# Variant Annotation, Filtering, Prioritization
## Sample usage of variantannotation and VarP packages as a pipeline for new epitope prediction

#### Author: C. Mazzaferro
#### Email: cmazzafe@ucsd.edu
#### Date: June 2016
 
## Outline of Notebook
<a id = "toc"></a>
1. <a href = "#background">Background</a>
2. <a href = "#setup">Set Up File and Libraries</a>
3. <a href = "#ANNOVAR">Run Annovar</a>
4. <a href = "#myvariant">Obtain data from myvariant.info</a>
5. <a href = "#filter">Variant Filtering & File Creation</a>
    * <a href = "#tumorvars">Rare Tumor Variant Filter</a>
    * <a href = "#diseasevars">Rare Diesease Variant Filter</a>
    * <a href = "#caddvars">CADD PHRED High Impact Variants</a>
    * <a href = "#own">Create Your Own Filter</a>
6. <a href = "#export">Export CSV and VCF files</a>

<a id = "background"></a>
## Background

This notebook will walk you through the steps of how variants coming from a VCF can be annotated efficiently and thoroughly using the package variantannotation. In particular, the package is aimed at providing a way of retrieving variant information using [ANNOVAR](http://annovar.openbioinformatics.org/en/latest/) and [myvariant.info](myvariant.info) and consolidating it in conveninent formats. It is well-suited for bioinformaticians interested in aggregating variant information into a single database for ease of use and to provide higher analysis capabities. 

The aggregation is performed specifically by structuring the data in lists of python dictionaries, with each variant being described by a multi-level dictionary. The choice was made due to the inconsistencies that exist between data availability, and the necessity to store their information in a flexible manner. Further, this specific format permits its parsing to a MongoDb instance (dictionaries are the python representation of JSON objects), which enables the user to efficiently store, query and filter such data. 

Finally, the package also has the added functionality to create csv and vcf files from MongoDB. The class Filters allows the user to rapidly query data that meets certain criteria as a list of documents, and the class FileWriter can transform such list into more widely accepted formats such as vcf and csv files. It should be noted that here, the main differential the package offers is the ability to write these files preserving all the annotation data. In the vcf files, for instance, outputs will have a 'Otherinfo' column where all the data coming from ANNOVAR and myvariant.info is condensed (while still preserving its structure). For vcf files, outputs will have around ~120-200 columns, depending on the amount of variant data that can be retrieved from myvvariant.info. 


**Notes on required software**

the following libraries will be install upon installing variantannotation:
- myvariant
- pysam
- pymongo
- pyvcf

Other libraries that are needed, but should natively e installed on most OS: 

- Watchdog
- Pandas
- Numpy

Further, a MongoDB database must be set up. Refer to the documentation page for more information. 
Similarly, ANNOVAR must be downloaded, alongside with its supporting databases (also listed on the documentation page).

<a id = "setup"></a>
## Import libraries and specify file paths

In [1]:
import os
import re
import sys
import vcf
import time
import pysam
import myvariant
import collections
import numpy as np
import pandas as pd
import seaborn as sns
from IPython.display import display, HTML

#variantannotation functions
from variantannotation import annotate_batch
from variantannotation import create_output_files
from variantannotation import myvariant_parsing_utils
from variantannotation import mongo_DB_export
from variantannotation import utilities
from variantannotation import MongoDB_querying

/Users/carlomazzaferro/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:1035: UserWarning: Duplicate key in file "/Users/carlomazzaferro/.matplotlib/matplotlibrc", line #2
  (fname, cnt))


In [2]:
ANNOVAR_PATH = '/data/annovar/'
FILE_NAMES = ['Tumor_RNAseq_variants.vcf', 'Tumor_targeted_seq.vcf', 'normal_targeted_seq.vcf', 'normal_blood_WGS.vqsr.vcf', 'somatic_mutect_old.vcf']
IN_PATH = '/data/ccbb_internal/interns/Carlo/test_vcf/'
OUT_PATH = '/data/ccbb_internal/interns/Carlo/test_vcf_out/'
vcf_file = IN_PATH

In [3]:
#Check if file paths are correctly pointing to the specified files.
for i in range(0, len(FILE_NAMES)):
    print IN_PATH+FILE_NAMES[i]

/data/ccbb_internal/interns/Carlo/test_vcf/Tumor_RNAseq_variants.vcf
/data/ccbb_internal/interns/Carlo/test_vcf/Tumor_targeted_seq.vcf
/data/ccbb_internal/interns/Carlo/test_vcf/normal_targeted_seq.vcf
/data/ccbb_internal/interns/Carlo/test_vcf/normal_blood_WGS.vqsr.vcf
/data/ccbb_internal/interns/Carlo/test_vcf/somatic_mutect_old.vcf


<a id = "ANNOVAR"></a>
## Run Annovar 

This will run ANNOVAR. A csv file named tumortargcsvout.hg19_multianno.csv will appear in the OUT_PATH specified. The csv file can then be processed and integrated with the data coming from myvariant.info. 
This command may take a some time to run (5-30 minutes for each file depending on file size).
To keep things simple, we can start by looking at one file only. Let's run annovar on it. In any case, if you have multiple files to work on, you can run them in parallel by running the block after the next one. 

In [ ]:
utilities.run_annovar(ANNOVAR_PATH, IN_PATH+FILE_NAMES[0], OUT_PATH)

Currently working on VCF file: Tumor_RNAseq_variants, field avinput
Currently working on VCF file: Tumor_RNAseq_variants, field variant_function
Currently working on VCF file: Tumor_RNAseq_variants, field exonic_variant_function
Currently working on VCF file: Tumor_RNAseq_variants, field hg19_tfbsConsSites
Currently working on VCF file: Tumor_RNAseq_variants, field hg19_cytoBand
Currently working on VCF file: Tumor_RNAseq_variants, field hg19_targetScanS
Currently working on VCF file: Tumor_RNAseq_variants, field hg19_genomicSuperDups
Currently working on VCF file: Tumor_RNAseq_variants, field hg19_gwasCatalog
Currently working on VCF file: Tumor_RNAseq_variants, field hg19_esp6500siv2_all_filtered
Currently working on VCF file: Tumor_RNAseq_variants, field hg19_esp6500siv2_all_dropped
Currently working on VCF file: Tumor_RNAseq_variants, field 2015_08_filtered
Currently working on VCF file: Tumor_RNAseq_variants, field 2015_08_dropped
Currently working on VCF file: Tumor_RNAseq_varian

In [ ]:
#Annovar runs as a subprocess on every file. They will run in parallel for speed up. 
for i in range(0, len(FILE_NAMES)):
    utilities.run_annovar(ANNOVAR_PATH, IN_PATH+FILE_NAMES[i], OUT_PATH)

#This serves to give a real-time feedback of the ANNOVAR progress and status. 


Specify the name and location of the csv file that ANNOVAR produces as output

In [4]:
filepath_out = '/data/ccbb_internal/interns/Carlo/test_vcf_out/'
filepath_in = '/data/ccbb_internal/interns/Carlo/test_vcf/'

#For safety, check the files in directory. Either run '!ls' here on iPython, or go to the directory and check 
#manually for existing files. There should be once csv file for every vcf file. 

VCF_FILE_NAME = 'Tumor_RNAseq_variants.vcf'
CSV_FILE_NAME = 'Tumor_RNAseq_variants.hg19_multianno.csv'

<a id = "myvariant"></a>
## Getting data from myvariant.info
The package offers 4 different methods to obtain variant data. Two of them require annovar, while the other two are based solely on the use of myvariant.info service. The latter can be used without having to worry about downloading and installing annovar databases, but it tends to return partial or no information for some variants. 

The different methods also enable the user to decide how the data will be parsed to MongoDB. 1 and 3 parse the data by chunks: the user specifies a number of variants (usually 1000), and the data from the vcf and csv files are parsed as soon as those 1000 variants are processed and integrated. This enables huge files to be processed without having to hold them in memory and potentially cause a Memory Overflow error. 

Methods 2 and 4, on the other hand, process the files on their entirety and send them to MongoDB at once. Well-suited for smaller files. See docs for more info. 

## Export data to MongoDB by chunks, iteratively. 

For this tutorial, we will use method #1. Data from annovar (as a csv file) will be obtained 1000 lines at a time, instead of attempting to parse and process an entire csv file at once.

As soon as you run the scripts from variantannotaiton, variant data will be retrieved from myvariant.info and the data will automatically be integrated and stored to MongoDB. Database and collection name should be specified, and there must be a running MongoDB connection. The script will set up a client to communicate between python (through pymongo) and the the database.

In general, the shell command:

`mongod --dbpath ../data/db`  

where data/db is the designated location where the data will be stored, will initiate MongoDB. After this, the script should store data to the directory automatically.
For pymongo, and more information on how to set up a Mongo Database: https://docs.mongodb.com/getting-started/python/

In [17]:
chunksize = 10000
step = 0

#Get variant list. Should always be the first step after running ANNOVAR
open_file = myvariant_parsing_utils.VariantParsing()

#Name Collections & DB. Change them to something appropriate. Each file should live in a collection
db_name = 'Variant_Prioritization_Workflow'

collection_name = 'Test_Tumor_RNAseq'

list_file = open_file.get_variants_from_vcf(filepath_in+VCF_FILE_NAME)
as_batch = annotate_batch.AnnotationMethods()
as_batch.by_chunks(list_file, chunksize, step, filepath_out+CSV_FILE_NAME, collection_name, db_name)

Converting columns to float ...
Processing knownGene info ...
Processing tfbsConsSites info ...
Processing genomicSuperDups info ...
Processing cytoBand info ...
Creating hgvs key ...
Processing genotype call info ...
Transforming to JSON from dataFrame
cleaning up...
Done
querying 1-1000...done.
querying 1001-2000...done.
querying 2001-3000...done.
querying 3001-4000...done.
querying 4001-5000...done.
querying 5001-6000...done.
querying 6001-7000...done.
querying 7001-8000...done.
querying 8001-9000...done.
querying 9001-10000...done.
Joining lists ...
Parsing to MongoDB ...
Step: 1 of 43
Converting columns to float ...
Processing knownGene info ...
Processing tfbsConsSites info ...
Processing genomicSuperDups info ...
Processing cytoBand info ...
Creating hgvs key ...
Processing genotype call info ...
Transforming to JSON from dataFrame
cleaning up...
Done
querying 1-1000...done.
querying 1001-2000...done.
querying 2001-3000...done.
querying 3001-4000...done.
querying 4001-5000...don

'Finished!'

<a id = "filter"></a>
## Variant Filtering & Output Files

Here we implement three different filters that allow for the retrieval of specific variants. The filters are implemented as MongoDB queries, and are designed to provie the user with a set of relevant variants. In case the user would like to define its own querying, a template is provided. 
The output of the queries is a list of dictionaries (JSON documents), where each dictionary contains data reltive to one variant. 

Further, the package allows the user to parse these variants into an annotated csv or vcf file. 
If needed, annotated, unfiltered vcf and csv files can also be created. They will have the same length (number of variants) as the original files, but will contain much more complete annotation data coming from myvariant.info and ANNOVAR databases. 

To create a csv file, just the filtered output is needed. To create an annotated vcf file, a tab indexed file (.tbi) file is needed (see comments in  section Create unfiltered annotated vcf and csv files at the end of this page). This can be created using tabix.  

First, the file needs to be compressed:

From the command line, running:

`bgzip -c input_file.vcf > input_file.vcf.gz`

returns `input_vcf_file.vcf.gz`

and running 

`tabix input_vcf_file.vcf.gz`

will return: `input_vcf_file.vcf.gz.tbi`



<a id = "tumorvars"></a>
## Specifying cancer-specifc rare variants

 - filter 1: ThousandGenomeAll < 0.05 or info not available
 - filter 2: ESP6500siv2_all < 0.05 or info not available
 - filter 3: cosmic70 information is present
 - filter 4: Func_knownGene is exonic, splicing, or both
 - filter 5: ExonicFunc_knownGene is not "synonymous SNV"
 - filter 6: Read Depth (DP) > 10

In [28]:
filepath = '/data/ccbb_internal/interns/Carlo'

In [9]:
#Create output files (if needed): specify name of files and path 
rare_cancer_variants_csv = filepath + "/tumor_rna_rare_cancer_vars_csv.csv"
rare_cancer_variants_vcf = filepath + "/tumor_rna_rare_cancer_vars_vcf.vcf"
input_vcf_compressed = filepath + '/test_vcf/Tumor_RNAseq_variants.vcf.gz'

#Apply filter.
filter_collection = MongoDB_querying.Filters(db_name, collection_name)
rare_cancer_variants = filter_collection.rare_cancer_variant()

#Crete writer object for filtered lists:
my_writer = create_output_files.FileWriter(db_name, collection_name)

#cancer variants filtered files
my_writer.generate_annotated_csv(rare_cancer_variants, rare_cancer_variants_csv)
my_writer.generate_annotated_vcf(rare_cancer_variants,input_vcf_compressed, rare_cancer_variants_vcf)

Variants found that match rarity criteria: 11


'Finished writing annotated, filtered VCF file'

## Read filtered vcf files

In [7]:
from VarP import utils

import sys
sys.path.append('/Users/carlomazzaferro/Documents/CCBB/neoantigen/VarP-master/')
reader_names = ['Tumor_RNA_Reader', 'Tumor_Targeted_Reader',
           'Normal_DNA_Reader', 'Normal_Blood_Reader',
           'Somatic_Mutect_Reader']   #bug fixed in source

path_to_files ='/Volumes/Seagate Backup Plus Drive/vcf_files/varcode_to_test/' 
myhandler = utils.HandleReaders(reader_names)

### Create a list of variant collections. See varcode's documentation for more info on this type of data

In [8]:
list_collections = myhandler.create_collection_from_readers(path_to_files)

print type(list_collections[4])
#Let's pick one of the collections to start working with
my_collection = list_collections[4]

OSError: [Errno 2] No such file or directory: '/Volumes/Seagate Backup Plus Drive/vcf_files/varcode_to_test/'

### Create desired outputs

In [33]:
list_coding_effects = myhandler.return_list_coding_effects(my_collection)   #list of coding effects
protein_list = myhandler.return_protein_list(list_coding_effects)           #list of proteins
dataframe = myhandler.return_dataframe(protein_list, list_coding_effects)   #dataframe

In [34]:
dataframe.head()

,prot,variants
0,MVSKLSQLQTELLAALLESGLSKEALIQALGEPGPYLLAGEGPLDK...,FrameShift(variant=chr12 g.121434630_121434631...
1,MVKSYLQQHNIPQREVVDTTGLNQSHLSQHLNKGTPMKTQKRAALY...,FrameShift(variant=chr12 g.121434630_121434631...
2,MEPSRALLGCLASAAAAAPPGEDGAGAGAEEEEEEEEAAAAVGPGE...,Deletion(variant=chr14 g.71275774_71275776delC...
3,MEPSRALLGCLASAAAAAPPGEDGAGAGAEEEEEEEEAAAAVGPGE...,Deletion(variant=chr14 g.71275774_71275776delC...
4,MEPSRALLGCLASAAAAAPPGEDGAGAGAEEEEEEEEAAAAVGPGE...,Deletion(variant=chr14 g.71275774_71275776delC...


In [8]:
#Fasta file
myhandler.generate_fasta_file(dataframe, path_to_files+'PEPTIDES.txt')

## New Epitope Prediction
#### Use epitope predict and netMHCpan

In [14]:
import seaborn as sns
import epitopepredict as ep
from epitopepredict import base, sequtils, analysis

sns.set_context("notebook", font_scale=1.4)
fastafile =  path_to_files+'PEPTIDES.txt'

#get data in DF format
zaire = sequtils.fasta_to_dataframe(fastafile)

P = base.get_predictor('netmhciipan')
#P1 = base.getPredictor('netmhciipan')
#P2 = base.getPredictor('netmhciipan')

savepath1 = 'netmhciipan'
#run prediction for several alleles and save results to savepath
alleles = ["HLA-DRB1*0101"]#, 
           
           #"HLA-DRB1*0108", "HLA-DRB1*0305", "HLA-DRB1*0401",
           #"HLA-DRB1*0404", "HLA-DRB3*0101", "HLA-DRB4*0104"]

P.predictProteins(zaire, length=11, alleles=alleles, path=savepath1)

predictions done for 18 proteins in 1 alleles
results saved to /Volumes/Seagate Backup Plus Drive/vcf_files/varcode_to_test/netmhciipan


In [19]:
P.load(path = savepath1)
P.summarize()
P.allele_summary()
P.protein_summary()

summary: 10029 peptides in 8 proteins and 1 alleles
                            peptide
name                               
Deletion(variant=chr14          686
Deletion(variant=chr16         3689
Deletion(variant=chr19         1447
Deletion(variant=chr9          1558
FrameShift(variant=chr12        364
Insertion(variant=chr6          803
Substitution(variant=chr19     1081
Substitution(variant=chr2       401


In [27]:
df.name.values[0]

'Deletion(variant=chr14'

In [24]:
#get promiscuous binders in at least 2 alleles above 5 percentile cutoff
name='ZEBOVgp3'
pb = P.promiscuousBinders(name=name,n=2,cutoff=5)
print 'promiscuous binders:'
print pb
print '--------------------------'
#binders sorted by median rank over all alleles, cutoff here is the median rank
print 'median ranked binders:'
mb = P.rankedBinders(name=name,cutoff=40)
print mb

ax=ep.utilities.venndiagram([pb.peptide, mb.peptide],
                      ['promiscuous<5%','median rank<20'])

no such protein name in binder data


AttributeError: 'NoneType' object has no attribute 'columns'

In [33]:
"""
mpk_files = os.listdir(path_to_files+'netmhciipan/')
P.data = pd.read_msgpack(mpk_files[0])
P1.data = pd.read_msgpack(mpk_files[1])
P2.data = pd.read_msgpack(mpk_files[2])
"""

In [36]:
mpk_files[0]

'Deletion(variant=chr14.mpk'

In [31]:
import types
"""
colormaps={'tepitope':'Greens','netmhciipan':'Oranges','iedbmhc2':'Pinks',
               'threading':'Purples','iedbmhc1':'Blues'}
colors = {'tepitope':'green','netmhciipan':'orange',
           'iedbmhc1':'blue','iedbmhc2':'pink','threading':'purple'}

def plotTracks(predictors, title='', alleles=2, width=900, height=None,
                seqdepot=None, bcell=None, exp=None, tools=True):
        Plot binding predictions in multiple alleles for a single protein.
        predictors: a dictionary of Predictor objects
        with their predicted binder data usually for a single protein. If data from  
        multiple proteins is provided the first one is used
        alleles: the minimum number of alleles for a binder to be shown
        

    from collections import OrderedDict
    from bokeh.palettes import Spectral3
    
    if type(predictors) is not types.ListType:
        predictors = [predictors]
    if tools == True:
        tools="xpan, xwheel_zoom, resize, hover, reset, save"
    else:
        tools=''
     
    #get title from the dataframe?
    
    alls=1
    n = alleles
    for p in predictors:
        alls += len(p.data.groupby('allele'))
    if height==None:
        height = 130+10*alls
    yrange = Range1d(start=0, end=alls+3)
    plot = Figure(title=title,title_text_font_size="11pt",plot_width=width,
                  plot_height=height, y_range=yrange,
                y_axis_label='allele',
                tools=tools,
                background_fill_color="#FAFAFA",
                toolbar_location="below")
    h=3
    '''if bcell != None:
        plotBCell(plot, bcell, alls)
    if seqdepot != None:
        plotAnnotations(plot,seqdepot)
    if exp is not None:
        plotExp(plot, exp)'''

    #lists for hover data
    #we plot all rects at once
    x=[];y=[];allele=[];widths=[];clrs=[];peptide=[]
    predictor=[];position=[];score=[];leg=[]
    l=80
    for pred in predictors:  
        m = pred.name
        print m, pred           
        df = pred.data        
        sckey = pred.scorekey
        pb = pred.getPromiscuousBinders(data=df,n=n)
        if len(pb) == 0:
            continue
        l = pred.getLength()
        grps = df.groupby('allele')
        alleles = grps.groups.keys()
        if len(pb)==0:
            continue
        c=colors[m]
        leg.append(m)

        for a,g in grps:
            b = pred.getBinders(data=g)             
            b = b[b.pos.isin(pb.pos)] #only promiscuous
            b.sort_values('pos',inplace=True)
            scores = b[sckey].values
            score.extend(scores)
            pos = b['pos'].values
            position.extend(pos)
            x.extend(pos+(l/2.0)) #offset as coords are rect centers
            widths.extend([l for i in scores])
            clrs.extend([c for i in scores])
            y.extend([h+0.5 for i in scores])
            alls = [a for i in scores]
            allele.extend(alls)
            peptide.extend(list(b.peptide.values))
            predictor.extend([m for i in scores])
            h+=1

    source = ColumnDataSource(data=dict(x=x,y=y,allele=allele,peptide=peptide,
                                    predictor=predictor,position=position,score=score))
    plot.rect(x,y, width=widths, height=0.8,
         #x_range=Range1d(start=1, end=seqlen+l),
         color=clrs,line_color='gray',alpha=0.7,source=source)
    
    hover = plot.select(dict(type=HoverTool))
    hover.tooltips = OrderedDict([
        ("allele", "@allele"),
        ("position", "@position"),
        ("peptide", "@peptide"),
        ("score", "@score"),
        ("predictor", "@predictor"),
    ])

    seqlen = pred.data.pos.max()+l
    plot.set(x_range=Range1d(start=0, end=seqlen+1))#, bounds=(0, seqlen+1)))
    plot.xaxis.major_label_text_font_size = "8pt"
    plot.xaxis.major_label_text_font_style = "bold"
    plot.ygrid.grid_line_color = None
    plot.yaxis.major_label_text_font_size = '0pt'
    plot.xaxis.major_label_orientation = np.pi/4        
    return plot

plot = plotTracks([P,P2],alleles=3)
show(plot)
"""
        
def plotStackedArea(pred, title=None):
    from bokeh.charts import Area,Line
    from bokeh.palettes import brewer, Spectral11
    
    tools="xpan, xwheel_zoom, resize, hover, reset, save"
    #plot = Figure(plot_width=800, plot_height=400)
        
    l = pred.getLength()        
    seqlen = pred.data.pos.max()+l   
    
    if title == None:
        title = list(pred.data.head(1).name)[0]
    #calculate plot data
    df = pred.data
    #b = pred.getBinders(data=df,n=n)
    #l = pred.getLength()
    grps = df.groupby('allele')
    #colors = brewer["Spectral"][len(grps)]
    #print t
    data = {}
    scores = []; pos=[]    
    for i,g in grps:
        #get running mean instead?        
        y = g.sort_values('pos')[pred.scorekey]                    
        y = y.clip(lower=0)
        #y = pd.rolling_mean(y, window=l, center=True).fillna(0)
        data[i] = y
        scores.extend(y.values)
        pos.extend(g.pos.values)
    
    #source = ColumnDataSource(data=dict(x=t.index,y=t.values))
    p = Area(data, title=title, legend="top_left", width=900, height=400, palette=Spectral11, 
                stack=True, xlabel='position', ylabel='score', tools=tools)
    
    grid = p.select(type=Grid)
    grid.grid_line_color = None
    p.set(x_range=Range1d(start=0, end=seqlen+1, bounds=(0, seqlen+1)))
            
    #glyphs = p.select(dict(type=GlyphRenderer))
    hover = p.select(dict(type=HoverTool))
    hover.tooltips = OrderedDict([
        ("x", "@x"),
        ("y", "@y"),
        #("score", "@scores"),
    ])
    return p

plot=plotStackedArea(P)
show(plot)

AttributeError: 'list' object has no attribute 'head'